<a href="https://colab.research.google.com/github/japjotsaggu/Fridays-with-Feynman/blob/main/Fridays_with_Feynman.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q langchain==0.0.150 pypdf tiktoken  transformers openai faiss-cpu InstructorEmbedding sentence_transformers

In [ ]:
import os
import openai
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain

In [ ]:
#loading combined pdf file (includes excerpts from Mr. Feynman's lectures, interviews and one of his books)
loader = PyPDFLoader("/content/final_merging.pdf")

#splitting our file into chunks
chunks = loader.load_and_split()

In [ ]:
#setting up the api environment
api_key = str(input("enter your api key: "))
os.environ["OPENAI_API_KEY"] = api_key

In [ ]:
#Getting embedding model from HuggingFace and creating a vector database out of our chunks
embeddings = HuggingFaceInstructEmbeddings(model_name = "hku-nlp/instructor-base")
db = FAISS.from_documents(chunks, embeddings)

In [ ]:
# Creating QA chain to integrate similarity search with user questions

chain = load_qa_chain(OpenAI(temperature=0), chain_type="stuff")

In [ ]:
from IPython.display import display
import ipywidgets as widgets

# Setting up chat history management - creating conversation chain that uses our db as retriver
qa = ConversationalRetrievalChain.from_llm(OpenAI(temperature = 0.1), db.as_retriever(), max_tokens_limit = 3000)

In [ ]:
#Bringing it all together - the actual widget
chat_history = []

def on_submit(_):
    query = input_box.value
    input_box.value = ""

    if query.lower() == 'exit':
        print("I enjoyed this chat, hope you did too! Come back soon to uncover more fascinating aspects of the world of Physics.")
        return

    result = qa({"question": query, "chat_history": chat_history})
    ans = result['answer']

    prompt = f'''Explain the following in detail as Richard Feynman in first person:
              "{ans}" '''

    response = openai.Completion.create(engine="text-davinci-002",  prompt=prompt, max_tokens = 500, api_key = api_key)

    final_ans = response.choices[0].text

    chat_history.append((query, final_ans))

    display(widgets.HTML(f'<b>User:</b> {query}'))
    display(widgets.HTML(f'<b><font color="blue">Mr.Feynman:</font></b> {final_ans}'))

print("It's a fine day for some FeynPhysics! Type 'exit' to stop.")

input_box = widgets.Text(placeholder='What fascinates you today?')
input_box.on_submit(on_submit)

display(input_box)